In [1]:
%load_ext autoreload
%autoreload 2

import sys
from datetime import datetime
import time
import random

import ipywidgets as widgets

import numpy as np
import keras
from keras.models import Sequential
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint

from sklearn.model_selection import train_test_split

from checkers_load import makeBatches, load_games, dataGenerator, getData
import checkers_models
from checkers_display import SimpleSquareBoardDisplay, MoveHeatmapDisplay

Using TensorFlow backend.


AttributeError: module 'keras_applications' has no attribute 'set_keras_submodules'

In [ ]:
games = load_games()
# games = games[:30]

X, Y = getData(games)

In [ ]:
b = SimpleSquareBoardDisplay()
display(b.fig)

In [ ]:
# Convert from (n,8,4,1)
#         to   (n,8,8,1) with empty squares.
# Should make convolution work better.

n = X.shape[0]

X2 = np.zeros((n,8,8,1))
#print(m.shape, m[::2, ::2].shape, m[1::2, 1::2].shape, x0[::2, :].shape, x0[1::2, :].shape)
X2[:, ::2, 1::2] = X[:, ::2, :]
X2[:, 1::2, ::2] = X[:, 1::2, :]

In [ ]:
# Test different ones
for i in range(1000):
    a = (X2[i] if i % 2 == 0 else -X2[i])
    b.update(a)
    time.sleep(1)

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X2, Y, test_size=0.10, random_state=42)

In [ ]:
filters = 128
layers = 0
model = checkers_models.literallyFuckingStupidModel(layers=1)

In [ ]:
m_name = "goConvNet-wfilters" + str(filters) + "-wlayers" + str(layers)


log_dir = 'logs/scalars/' + m_name + "-" + datetime.now().strftime("%m%d-%H%M%S")
tf_callback = keras.callbacks.TensorBoard(log_dir=log_dir)
print("!!!!!!!!!!!!! Logging to", log_dir, "!!!!!!!!!!!!!!!!!!")

batch_s = 256
epochs = 20

model.fit(X_train, Y_train,
          batch_size=batch_s,
          epochs=epochs,
          validation_split=0.1,
          callbacks=[tf_callback])

# Save
f_name = m_name + '.h5'
print("Saving", f_name)
model.save(f_name)

In [ ]:
#model = keras.models.load_model(f_name)

figs = []

board_d = SimpleSquareBoardDisplay()
figs.append(board_d.fig)

move_d = MoveHeatmapDisplay()
figs.append(move_d.fig)

pred_d = MoveHeatmapDisplay()
figs.append(pred_d.fig)

display(widgets.HBox(figs))

In [ ]:
ix = random.randrange(0,10000)

#for ix in range(30):
x_i, y_i = X_test[ix], Y_test[ix]

x = np.expand_dims(x_i, axis=0)
y_pred = model.predict([x])[0]

#ymax = np.amax(y_pred)
#y_pred = y_pred / ymax


board_d.update(x_i)
move_d.update(y_i)
pred_d.update(y_pred)